In [20]:
import eikon as ek
import refinitiv.dataplatform as rdp
ek.set_app_key('04c0e3f661bd49348d69c3aabedb8c0108cfd1e2')

df, err = ek.get_data(
    instruments = ['GOOG.O','MSFT.O', 'FB.O'], 
    fields = ['TR.Revenue']
)
display(df)


,Instrument,Revenue
0,GOOG.O,282836000000
1,MSFT.O,198270000000
2,FB.O,116609000000


In [1]:

df, err = ek.get_data(
    instruments = ['GOOGL.O'],
    fields = [
        'TR.CompanyMarketCapitalization',
        'TR.Revenue',
        'TR.NetProfitMean',
        'TR.PE',
        'TR.PriceToSalesPerShare'
    ],
    parameters = {
        'SDate': '1D',
        'Curn': 'USD'
    }
)

display(df)

NameError: name 'ek' is not defined

In [4]:
import eikon as ek
import pandas as pd
import refinitiv.dataplatform as rdp
import datetime as dt


#set app key to identify the application on Refinitiv Platform
ek.set_app_key('04c0e3f661bd49348d69c3aabedb8c0108cfd1e2')

#a RIC that you want to get time series
RIC = 'AAPL.OQ'

#start and end date; it is in format yyyy-mm-dd
start_date = '2018-01-02'
now = dt.datetime.now()
start = now - dt.timedelta(days = 365*25)

#get time series till the last market close date
df = ek.get_timeseries(RIC, 'CLOSE', interval='daily', start_date=start, end_date=now)

#initialising demo of generating timeseries for list of stock tickers
RICs = ['AAPL.OQ','MSFT.OQ','2222.SE','GOOGL.OQ','AMZN.OQ']
dfs = list()
for ticker in RICs:
    dfs.append(ek.get_timeseries(ticker,'CLOSE',interval='daily',start_date=start,end_date=now))

display(dfs[0])

AAPL.OQ,CLOSE
Date,
2011-05-02,12.36713
2011-05-03,12.435702
2011-05-04,12.496059
2011-05-05,12.389988
2011-05-06,12.380702
...,...
2023-03-27,158.28
2023-03-28,157.65
2023-03-29,160.77


In [218]:
import os
dataPath = os.path.abspath(os.path.join(os.path.dirname((os.path.curdir.__dir__()[0])), '..', 'Data')) #goes up a dir to equity eagle then into Data dir
#os.makedirs(dataPath + "\{}".format("MSFT.OQ")) 
# df = ek.get_timeseries(ticker,'CLOSE',interval='daily',start_date=start,end_date=now)
# df.to_csv(path_or_buf = dataPath + "\{}\Prices.csv".format(ticker))
df,err = ek.get_data('PEERS("LVMH.PA")'.format(ticker),
    [
    'TR.DividendYield','TR.F.DivPayoutRatioPct/100','TR.F.MktCap(Period=FY0)'
    ])
#df['Price / EPS (SmartEstimate ®)'] = pd.to_numeric(df['Price / EPS (SmartEstimate ®)'],errors='coerce') #remove nan string in starMine col
#df = df.dropna(axis=0,how='any')
display(df)
'''
fundamental data:
=company overview: description, founded, employees, website
=stock graph
=volatility vs peers average
=revenue
=market cap
=earnings
=PE
=PS
=All peers metrics
=Cost of revenue
=current dividen yield
=payout ratio
'''

,None,TR.DIVIDENDYIELD(CURN=USD),TR.F.DIVPAYOUTRATIOPCT/100,TR.F.MKTCAP(PERIOD=FY0)
0,"PEERS(""LVMH.PA"")",<NA>,<NA>,<NA>


'\nfundamental data:\n=company overview: description, founded, employees, website\n=stock graph\n=volatility vs peers average\n=revenue\n=market cap\n=earnings\n=PE\n=PS\n=All peers metrics\n=Cost of revenue\n=current dividen yield\n=payout ratio\n'

In [231]:
from IPython.core.display import display, HTML
df = ek.get_news_headlines(query='PresetTopic:[Significant News: All] AND R:AAPL.O AND Language:LEN')
df['HTML'] = df['storyId'].apply(ek.get_news_story)
display(HTML(ek.get_news_story(df['storyId'][2])))

C:\Users\paulv\AppData\Local\Temp\ipykernel_14588\695389811.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [207]:
from bs4 import BeautifulSoup
from textblob import TextBlob
import numpy as np
df['Polarity'] = np.nan
df['Subjectivity'] = np.nan
df['Score'] = np.nan

for idx, storyId in enumerate(df['storyId'].values):
    newsText = ek.get_news_story(storyId) #get the news story
    if newsText:
        soup = BeautifulSoup(newsText,"lxml") #create a BeautifulSoup object from our HTML news article
        sentA = TextBlob(soup.get_text()) #pass the text only article to TextBlob to anaylse
        df['Polarity'].iloc[idx] = sentA.sentiment.polarity #write sentiment polarity back to df
        df['Subjectivity'].iloc[idx] = sentA.sentiment.subjectivity #write sentiment subjectivity score back to df
        if sentA.sentiment.polarity >= 0.05: # attribute bucket to sentiment polartiy
            score = 'positive'
        elif  -.05 < sentA.sentiment.polarity < 0.05:
            score = 'neutral'
        else:
            score = 'negative'
        df['Score'].iloc[idx] = score #write score back to df
df.head()

C:\Users\paulv\AppData\Local\Temp\ipykernel_14588\2206903417.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Polarity'].iloc[idx] = sentA.sentiment.polarity #write sentiment polarity back to df
C:\Users\paulv\AppData\Local\Temp\ipykernel_14588\2206903417.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Subjectivity'].iloc[idx] = sentA.sentiment.subjectivity #write sentiment subjectivity score back to df
C:\Users\paulv\AppData\Local\Temp\ipykernel_14588\2206903417.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http

,versionCreated,text,storyId,sourceCode,Polarity,Subjectivity,Score
2023-03-31 22:47:30.000,2023-03-31 22:47:30+00:00,Apple wins reversal of $502 mln VirnetX patent...,urn:newsml:newswire.refinitiv.com:20230331:nL1...,NS:RTRS,0.041667,0.462500,neutral
2023-03-31 22:29:52.569,2023-03-31 22:35:18.620000+00:00,"Refinitiv Newscasts - Stocks rally, Nasdaq pos...",urn:newsml:newswire.refinitiv.com:20230331:nRT...,NS:RTRS,0.075000,0.325000,positive
2023-03-31 20:00:25.000,2023-03-31 20:00:25+00:00,"US STOCKS-Nasdaq posts best qtr since 2020, la...",urn:newsml:newswire.refinitiv.com:20230331:nL1...,NS:RTRS,0.102759,0.395794,positive
2023-03-31 14:00:00.000,2023-03-31 19:54:46+00:00,CORRECTED-GM plans to phase out Apple CarPlay ...,urn:newsml:newswire.refinitiv.com:20230331:nL1...,NS:RTRS,0.120044,0.341345,positive
2023-03-31 16:38:22.000,2023-03-31 18:45:19+00:00,UPDATE 2-Apple wins appeal against UK's decisi...,urn:newsml:newswire.refinitiv.com:20230331:nL8...,NS:RTRS,0.157292,0.539062,positive


In [211]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd

data = [[0.044,0,0.025],[0.046,0,0.055]]
df = pd.DataFrame(data, columns=['x','y','axis'])
display(df)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=[0.044,0.046], y=[0,0], mode='markers+text', marker_size=30, text=['Ticker','Industry'], textposition='top center', marker_symbol = 'star-diamond',marker_color="midnightblue",
))
fig.add_annotation(x=0.025
    , y=0
    , text=f'Low'
    , showarrow=True
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=0
    , ay=0
    , font=dict(size=12, color="white", family="Courier New, monospace")
    , align="left"
    , bordercolor='black'
    , borderwidth=2
    , bgcolor="black"
    , opacity=1)
fig.add_annotation(x=0.055
    , y=0
    , text=f'High'
    , showarrow=True
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=0
    , ay=0
    , font=dict(size=12, color="white", family="Courier New, monospace")
    , align="left"
    , bordercolor='black'
    , borderwidth=2
    , bgcolor="black"
    , opacity=1)
fig.update_xaxes(showgrid=False, range=[0.025,0.055])
fig.update_yaxes(showgrid=False,
                 zeroline=True, zerolinecolor='green', zerolinewidth=20,
                 showticklabels=False, range=[0,0])
fig.update_layout(height=500, plot_bgcolor='white')
fig.show()

,x,y,axis
0,0.044,0,0.025
1,0.046,0,0.055


In [213]:
import plotly.graph_objs as go
import numpy as np

# Generate random data for the heatmap
x_values = np.arange(10)
y_values = np.arange(10)
z_values = np.random.rand(10,10)

# Create heatmap trace with no colorscale
heatmap = go.Heatmap(
    x=x_values,
    y=y_values,
    z=z_values,
    colorscale='Greys',
    showscale=False
)

# Define layout of the plot
layout = go.Layout(
    xaxis=dict(
        side='bottom',
        ticks='',
        ticklen=0,
        tickfont=dict(color='black', size=14),
        showticklabels=True,
        showgrid=False,
        line=dict(colorscale='Viridis', color=np.arange(len(x_values)))
    ),
    yaxis=dict(
        ticks='',
        ticklen=0,
        showticklabels=False,
        showgrid=False
    ),
    margin=dict(t=50),
)

# Create figure object and plot it
fig = go.Figure(data=[heatmap], layout=layout)
fig.show()
pd.re

ValueError: Invalid property specified for object of type plotly.graph_objs.layout.XAxis: 'line'

Did you mean "side"?

    Valid properties:
        anchor
            If set to an opposite-letter axis id (e.g. `x2`, `y`),
            this axis is bound to the corresponding opposite-letter
            axis. If set to "free", this axis' position is
            determined by `position`.
        automargin
            Determines whether long tick labels automatically grow
            the figure margins.
        autorange
            Determines whether or not the range of this axis is
            computed in relation to the input data. See `rangemode`
            for more info. If `range` is provided, then `autorange`
            is set to False.
        autotypenumbers
            Using "strict" a numeric string in trace data is not
            converted to a number. Using *convert types* a numeric
            string in trace data may be treated as a number during
            automatic a